In [8]:
import sys
sys.path.append("tests/helper")

In [9]:
from mosek.fusion import SolutionError

In [11]:
import random
from typing import Callable, Tuple
from copy import deepcopy

import numpy as np
import pytest
from gl0learn import fit, synthetic, Penalty
from gl0learn.metrics import nonzeros, pseudo_likelihood_loss
from gl0learn.opt import MIO_mosek
from gl0learn.gl0learn_core import check_coordinate_matrix
from gl0learn.utils import triu_nnz_indicies
from hypothesis import given, settings, HealthCheck, assume, note
from hypothesis.strategies import just, booleans, floats, integers, random_module

from utils import (
    sample_from_cov,
    overlap_covariance_matrix,
    is_scipy_installed,
    is_mosek_installed,
    make_bisect_func,
    random_penalty,
    random_penalty_values,
    top_n_triu_indicies_by_abs_value,
)

In [16]:
MAX_OVERLAPS = 6

@given(
    n=integers(3, 1000),
    p=integers(3, 10),
    overlaps=integers(1, MAX_OVERLAPS - 1),
    module=random_module(),
    lXs=random_penalty_values(
        penalty_strategies=random_penalty(l0=just(True), l1=just(False), l2=just(True)),
        values_strategies={"l0": floats(0.1, 10), "l2": floats(0.1, 10)},
    ),
)
@settings(max_examples=250, deadline=None)
def test_cd_learns_mio_results_from_support(algorithm, n, p, module, overlaps, lXs):
    from gl0learn.opt import MIO_mosek

    theta_truth = overlap_covariance_matrix(
        p=p,
        seed=module.seed,
        max_overlaps=overlaps,
        decay=1 - np.exp(overlaps - MAX_OVERLAPS),
    )

    assume(all(np.linalg.eigvalsh(theta_truth) > 0))
    x = sample_from_cov(n=n, cov=theta_truth)

    _, _, _, _, y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

    m = np.max(np.abs(theta_truth * (1 - np.eye(p))))
    int_tol = 1e-4

    MIO_results = MIO_mosek(y=y, m=m, **lXs, int_tol=int_tol)
    active_set = triu_nnz_indicies(MIO_results.theta_hat)
    cd_results = fit(
        y,
        **lXs,
        scale_x=False,
        theta_init=None,
        max_iter=1000,
        algorithm=algorithm,
        active_set=active_set,
        super_active_set=active_set,
        max_active_set_ratio=1.0
    )

    penalty = Penalty(**lXs)
    MIO_loss = pseudo_likelihood_loss(
        y, np.array(MIO_results.theta_hat), penalty, active_set=active_set
    )
    cd_loss = pseudo_likelihood_loss(
        y, np.array(cd_results.theta), penalty, active_set=active_set
    )

    assert cd_loss <= MIO_loss + int_tol * abs(MIO_loss)

In [17]:
class RandomSeeder:
    
    def __init__(self, seed: int):
        self.seed = seed

In [18]:
# n = 40
# p = 4
# module = RandomSeeder(29)
# overlaps = 2
# lXs = {'l0': 0.01, 'l2': 0.01}

# n = 50
# p = 3
# module = RandomSeeder(0)
# overlaps = 1
# lXs = {'l0': 0.5452059022532182, 'l2': 0.1}



In [19]:
#TODO, Seem to be operating on the cov of X, not y... How is this happening?

test_cd_learns_mio_results_from_support(algorithm="CD")

gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2


/Users/tnonet/opt/anaconda3/envs/gl0learn_pypkg/lib/python3.9/site-packages/gl0learn/utils.py:150: UserWarning: gl0learn requires Fortran-style array. Array , provided scope name = 'x', detected that is C-style. Generating copy that is Fortran-style. To decrease memory usage, ensure array is provided as Fortran-style and no copy will be made. See: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flags.html.
  warnings.warn(
/Users/tnonet/opt/anaconda3/envs/gl0learn_pypkg/lib/python3.9/site-packages/gl0learn/utils.py:150: UserWarning: gl0learn requires Fortran-style array. Array , provided scope name = 'x', detected that is C-style. Generating copy that is Fortran-style. To decrease memory usage, ensure array is provided as Fortran-style and no copy will be made. See: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flags.html.
  warnings.warn(


gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0Lea

/Users/tnonet/opt/anaconda3/envs/gl0learn_pypkg/lib/python3.9/site-packages/gl0learn/utils.py:150: UserWarning: gl0learn requires Fortran-style array. Array , provided scope name = 'x', detected that is C-style. Generating copy that is Fortran-style. To decrease memory usage, ensure array is provided as Fortran-style and no copy will be made. See: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flags.html.
  warnings.warn(


gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2


...

gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2


/Users/tnonet/opt/anaconda3/envs/gl0learn_pypkg/lib/python3.9/site-packages/gl0learn/utils.py:150: UserWarning: gl0learn requires Fortran-style array. Array , provided scope name = 'x', detected that is C-style. Generating copy that is Fortran-style. To decrease memory usage, ensure array is provided as Fortran-style and no copy will be made. See: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flags.html.
  warnings.warn(


gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2


/Users/tnonet/opt/anaconda3/envs/gl0learn_pypkg/lib/python3.9/site-packages/gl0learn/utils.py:150: UserWarning: gl0learn requires Fortran-style array. Array , provided scope name = 'x', detected that is C-style. Generating copy that is Fortran-style. To decrease memory usage, ensure array is provided as Fortran-style and no copy will be made. See: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flags.html.
  warnings.warn(


gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2


In [83]:
def test_cd_keeps_mio_results(n, p, module, overlaps, lXs):
    theta_truth = overlap_covariance_matrix(
        p=p, seed=module.seed, max_overlaps=overlaps, decay=1 - np.exp(overlaps - 6)
    )

    assume(all(np.linalg.eigvalsh(theta_truth) > 0))
    x = sample_from_cov(n=n, cov=theta_truth)

    _, _, _, _, y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

    m = np.max(np.abs(theta_truth * (1 - np.eye(p))))
    int_tol = 1e-9

    MIO_results = MIO_mosek(y=y, m=m, **lXs, int_tol=int_tol, mio_gap=int_tol, max_time=10)
    print(MIO_results)
    cd_results = fit(
        y,
        **lXs,
        scale_x=False,
        theta_init=MIO_results.theta_hat,
        active_set=0.0,
        super_active_set=0.0,
        max_active_set_ratio=1.0
    )

    np.testing.assert_array_equal(MIO_results.theta_hat, cd_results.theta)

In [8]:
n = 300
p = 3
module = RandomSeeder(0)
overlaps = 1
lXs = {'l0': 0.1, 'l2': 0.1}
penalty = Penalty(**lXs)

In [9]:
theta_truth = overlap_covariance_matrix(
        p=p, seed=module.seed, max_overlaps=overlaps, decay=1 - np.exp(overlaps - 6)
)

assume(all(np.linalg.eigvalsh(theta_truth) > 0))
x = sample_from_cov(n=n, cov=theta_truth)

_, _, _, _, y, _ = synthetic.preprocess(x, assume_centered=False, cholesky=True)

m = np.max(np.abs(theta_truth * (1 - np.eye(p))))
int_tol = 1e-9

MIO_results = MIO_mosek(y=y, m=m, **lXs, int_tol=int_tol, mio_gap=int_tol, max_time=10)
print(MIO_results)
cd_results = fit(
    y,
    **lXs,
    scale_x=False,
    theta_init=MIO_results.theta_hat,
    active_set=0.0,
    super_active_set=0.0,
    max_active_set_ratio=1.0
)

MIOResults(theta_hat=array([[1.0470939 , 0.        , 0.        ],
       [0.        , 1.00160605, 0.49999999],
       [0.        , 0.49999999, 1.06869336]]), z=array([[0., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]]), t=array([0.99999638, 0.96566702, 0.96566351]), s=array([0.        , 0.        , 0.        , 0.        , 0.24999999,
       0.        ]), lg=array([0.04601861, 0.00160476, 0.06643674]), residuals=array([ 1.02327421, -0.01216462,  0.01917952,  0.        ,  0.8610878 ,
       -0.03169435,  0.        ,  0.47497133,  1.01519741]), objective=2.9422667976142503, elapsed=0.07879899100000021, upper_bound=2.94226679761425, lower_bound=2.94226679761425, gap=0.0, status=ProblemStatus.PrimalFeasible)
gL0LearnFit 1
gL0LearnFit 2
gL0LearnFit 2
fit 1
current_iter: 0 cur_objective = 2.94227
fit loop0
theta[0, 1] = 0
new_theta[0, 1] = 0
theta[0, 2] = 0
new_theta[0, 2] = -0
theta[1, 2] = 0.5
new_theta[1, 2] = 0.507356
current_iter: 1 cur_objective = 2.94208
fit loop1
theta[0, 1] = 0


/Users/tnonet/opt/anaconda3/envs/gl0learn_pypkg/lib/python3.9/site-packages/gl0learn/utils.py:145: UserWarning: gl0learn requires Fortran-style array. Array , provided scope name = 'theta_init', detected that is C-style. Generating copy that is Fortran-style. To decrease memory usage, ensure array is provided as Fortran-style and no copy will be made. See: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flags.html.
  warnings.warn(


In [10]:
np.uint64

numpy.uint64

In [11]:
np.triu_indices(p, k=1)

(array([0, 0, 1]), array([1, 2, 2]))

In [12]:
check_coordinate_matrix(np.asarray(np.triu_indices(p, k=1), order="C", dtype=np.uint64).T, True, True)

True

In [13]:
active_set = np.asarray(np.triu_indices(p, k=1), order="C", dtype=np.uint64).T

In [14]:
check_coordinate_matrix(active_set, True, True)

True

In [15]:
pseudo_likelihood_loss(y, np.array(MIO_results.theta_hat), penalty, active_set=active_set)

2.942266797905327

In [16]:
pseudo_likelihood_loss(y, np.array(cd_results.theta), penalty, active_set=active_set)

2.9420494724702024